In [2]:
import pandas as pd
import numpy as np
import scipy
import sklearn

path = "./data/anonymous-msweb.test"
raw_data = pd.read_csv(path,header=None,skiprows=7)
raw_data.shape
raw_data.head()

,0,1,2,3,4
0,A,1277,1,NetShow for PowerPoint,/stream
1,A,1253,1,MS Word Development,/worddev
2,A,1109,1,TechNet (World Wide Web Edition),/technet
3,A,1038,1,SiteBuilder Network Membership,/sbnmember
4,A,1205,1,Hardware Supprt,/hardwaresupport


In [3]:
#creating user profile
user_activity = raw_data.loc[raw_data[0] != "A"]
user_activity.columns = ['category','value','vote','desc','url']
user_activity = user_activity[['category','value']]

In [4]:
user_activity.groupby('category').count()


,value
category,
C,5000
V,15191


In [5]:
len(user_activity.loc[user_activity['category'] =="C"].value.unique())

5000

In [6]:
len(user_activity.loc[user_activity['category'] =="V"].value.unique())
user_activity.index

Int64Index([  294,   295,   296,   297,   298,   299,   300,   301,   302,
              303,
            ...
            20475, 20476, 20477, 20478, 20479, 20480, 20481, 20482, 20483,
            20484],
           dtype='int64', length=20191)

In [7]:
tmp = 0
nextrow = False
print(user_activity.index)
lastindex = user_activity.index[len(user_activity)-1]
lastindex

Int64Index([  294,   295,   296,   297,   298,   299,   300,   301,   302,
              303,
            ...
            20475, 20476, 20477, 20478, 20479, 20480, 20481, 20482, 20483,
            20484],
           dtype='int64', length=20191)


20484

In [8]:
for index,row in user_activity.iterrows():
    if(index <= lastindex ):
        if(user_activity.loc[index,'category'] == "C"):
            tmp = 0
            #add += 1
            #user_activity.loc[index,'chunk'] += add
            userid = user_activity.loc[index,'value']
            user_activity.loc[index,'userid'] = userid
            user_activity.loc[index,'webid'] = userid
            tmp = userid
            nextrow = True            
        elif(user_activity.loc[index,'category'] != "C" and nextrow == True):
                #user_activity.loc[index,'chunk'] += add
                webid = user_activity.loc[index,'value']
                user_activity.loc[index,'webid'] = webid
                user_activity.loc[index,'userid'] = tmp
                if(index != lastindex and user_activity.loc[index+1,'category'] == "C"):
                    nextrow = False

In [9]:
user_activity_v = user_activity[user_activity['category'] == "V" ]
user_activity_v = user_activity_v[['userid','webid']]
print(user_activity_v.index)
user_activity_sort = user_activity_v.sort_values('webid', ascending=True)


Int64Index([  295,   296,   297,   299,   300,   301,   303,   304,   305,
              306,
            ...
            20471, 20473, 20475, 20476, 20478, 20479, 20480, 20481, 20483,
            20484],
           dtype='int64', length=15191)


In [10]:
user_activity_sort['userid'].unique().shape[0]


5000

In [11]:
user_activity_sort['webid'].unique().shape[0]

236

In [12]:
sLength = len(user_activity_sort['webid'])
user_activity_sort['rating'] = pd.Series(np.ones((sLength,)), index=user_activity_v.index)
ratmat = user_activity_sort.pivot(index='userid', columns='webid', values='rating').fillna(0)
print(ratmat.shape)
ratmat = ratmat.to_dense().as_matrix()
user_activity_sort.shape

(5000, 236)


(15191, 3)

In [13]:
items = raw_data.loc[raw_data[0] == "A"]
items.columns = ['record','webid','vote','desc','url']
items = items[['webid','desc']]
items2 = items[items['webid'].isin(user_activity_v['webid'].tolist())]
items_sort = items2.sort_values('webid', ascending=True)
items['webid'].unique().shape[0]


294

In [14]:
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(stop_words ="english",max_features = 100,ngram_range= (0,3),sublinear_tf =True)
x = v.fit_transform(items_sort['desc'])
itemprof = x.todense()
print(ratmat)
itemprof.shape

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


(236, 100)

In [19]:
#dot product
from scipy import linalg, dot
userprof = dot(ratmat,itemprof)/linalg.norm(ratmat)/linalg.norm(itemprof)
userprof.shape

(5000, 100)

In [16]:
#cosine similarity between  userprofile an item profile
import sklearn.metrics
similarityCalc = sklearn.metrics.pairwise.cosine_similarity(userprof, itemprof, dense_output=True)
print(similarityCalc.shape)
#covert the rating to binary format
final_pred= np.where(similarityCalc>0.6, 1, 0)
print(final_pred[943])
indexes_of_user = np.where(final_pred[943] == 1)
print(indexes_of_user)

(5000, 236)
[1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 0
 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 0
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 1
 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0
 0 1 1 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0
 0 0 0 1 1 0 0 0 1 0 0 0 0 0]
(array([  0,   3,   5,   6,  18,  21,  23,  33,  42,  43,  45,  52,  55,
        57,  58,  60,  65,  70,  71,  72,  78,  79,  83,  85,  91,  93,
        97, 101, 102, 103, 104, 105, 108, 109, 112, 118, 120, 122, 131,
       133, 135, 136, 137, 140, 143, 147, 148, 150, 159, 160, 165, 166,
       168, 172, 173, 175, 176, 181, 183, 186, 187, 189, 192, 194, 196,
       197, 203, 204, 209, 212, 213, 219, 220, 225, 226, 230]),)


In [21]:
userprof[indexes_of_user].shape

(76, 100)